In [1]:
import numpy as np
import pandas as pd
import re
from time import time

from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [2]:
# task 1

train_df=pd.read_csv("DS3_C3_S2_TweetTrain_Data_Challenge.csv")
test_df=pd.read_csv("DS3_C3_S2_TweetTest_Data_Challenge.csv")

In [3]:
train_df

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km�),Density (P/Km�)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26
...,...,...,...,...,...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,night,31-45,Ghana,31072940,227540.0,137
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,morning,46-60,Greece,10423054,128900.0,81
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,noon,60-70,Grenada,112523,340.0,331
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,night,70-100,Guatemala,17915568,107160.0,167


In [4]:
test_df

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km�),Density (P/Km�)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797,27400.0,105
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044,2381740.0,18
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265,470.0,164
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272,1246700.0,26
...,...,...,...,...,...,...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative,noon,21-30,Nicaragua,6624554,120340.0,55
3530,416863ce47,All alone in this old house again. Thanks for...,positive,night,31-45,Niger,24206644,1266700.0,19
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative,morning,46-60,Nigeria,206139589,910770.0,226
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive,noon,60-70,North Korea,25778816,120410.0,214


In [5]:
train_df.isnull().sum()

textID              0
text                1
selected_text       1
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km�)     0
Density (P/Km�)     0
dtype: int64

In [6]:
train_df.dropna(inplace=True)

In [7]:
test_df.isnull().sum()

textID              0
text                0
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km�)     0
Density (P/Km�)     0
dtype: int64

In [8]:
# task 2

traintext=train_df.loc[:,['text','sentiment']]
testtext=test_df.loc[:,['text','sentiment']]

In [9]:
import string
def text_process(text):
    nopunc=[char for char in text if char not in string.punctuation] 
    nopunc=''.join(nopunc) 
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [10]:
def listToString(s):
    str1="" 
    return(str1.join(s))

In [11]:
traintext['text']=traintext['text'].apply(text_process)

In [12]:
traintext['text']=traintext['text'].apply(listToString)

In [13]:
testtext['text']=testtext['text'].apply(text_process)

In [14]:
testtext['text']=testtext['text'].apply(listToString)

In [15]:
x_train=traintext['text']
y_train=traintext['sentiment']

In [16]:
x_test=testtext['text']
y_test=testtext['sentiment']

In [17]:
# task 3

tfidf=TfidfVectorizer()
oj1=tfidf.fit_transform(x_train)
oj2=tfidf.transform(x_test)

lr=LogisticRegression(max_iter=200,verbose=1)
ovr=OneVsRestClassifier(lr)

ovr.fit(oj1,y_train)
y_pred=ovr.predict(oj2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [18]:
# task 4

data = pd.DataFrame({"Actual":y_test[:30],"Predicted":y_pred[:30]})
data

,Actual,Predicted
0,neutral,neutral
1,positive,neutral
2,negative,neutral
3,positive,neutral
4,positive,neutral
5,positive,neutral
6,negative,neutral
7,negative,neutral
8,neutral,neutral
9,neutral,neutral


In [19]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

    negative       0.89      0.02      0.03      1001
     neutral       0.41      1.00      0.58      1430
    positive       1.00      0.04      0.07      1103

    accuracy                           0.42      3534
   macro avg       0.77      0.35      0.23      3534
weighted avg       0.73      0.42      0.27      3534



In [20]:
confusion_matrix(y_test , y_pred)

array([[  17,  984,    0],
       [   2, 1428,    0],
       [   0, 1064,   39]], dtype=int64)